#Importar bibliotecas

In [2]:
import pandas as pd
from sklearn.utils import shuffle
from pandas import DataFrame
import numpy as np
from IPython.core.debugger import set_trace
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [3]:
from sklearn.ensemble import RandomForestClassifier

#Instalar e importar MongoDB

In [4]:
! python -m pip install pymongo==3.7.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 628 kB 22.0 MB/s 
  Created wheel for pymongo: filename=pymongo-3.7.2-cp38-cp38-linux_x86_64.whl size=415785 sha256=1d5f15adf5e66e0d73f7befe10c69c1caf58bf898e130ec9ba5703694f35bc3f
  Stored in directory: /root/.cache/pip/wheels/28/62/b5/ede9674d1415d2c15c3e805e6cc7debfcdf380105da0887776
Successfully built pymongo
  Attempting uninstall: pymongo
    Found existing installation: pymongo 4.3.3
    Uninstalling pymongo-4.3.3:
      Successfully uninstalled pymongo-4.3.3


In [5]:
! python -m pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import datetime                           
import pymongo
from pymongo import MongoClient
# uri (uniform resource identifier) define os parêmtros de conexão com o banco de dados
uri = '' # IMPORTANTE: colocar aqui a url do banco de dados mongodb
# conecta à coleção alvo do mongoose
cluster = MongoClient( uri )
db = cluster["inteli-iot"]
collection = db["locationsTrainingData"]

#Tratamento de dados

Levando em consideração que para fornecer a localização iremos usar a potência de sinal, é necessário saber qual a potência que cada roteador irradia, então será usado o macAddress. Para isso, tratamos os dados coletados no banco, separando cada macAddress distinto sua própria coluna, ou seja, cada macAddress será uma feature, e as potências de sinal dele em cada sala.

In [7]:
df = pd.DataFrame(list(collection.find({'room': {'$exists': True}})))
df

,_id,deviceId,room,signals,battery,createdAt,updatedAt,__v
0,639a1a8b639dabb65f7a1fd9,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'F6:12:FA:E2:55:04', 'strength...",60.0,2022-12-14 18:48:43.522,2022-12-14 18:48:43.522,0
1,639a1afe639dabb65f7a1ffd,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'FE:F6:3A:68:98:A3', 'strength...",60.0,2022-12-14 18:50:38.387,2022-12-14 18:50:38.387,0
2,639a1b13639dabb65f7a201a,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'FE:F6:3A:68:98:A3', 'strength...",60.0,2022-12-14 18:50:59.473,2022-12-14 18:50:59.473,0
3,639a1b24639dabb65f7a2045,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'FC:5C:45:00:57:48', 'strength...",60.0,2022-12-14 18:51:16.251,2022-12-14 18:51:16.251,0
4,639a1b40639dabb65f7a207a,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'FC:5C:45:00:57:48', 'strength...",60.0,2022-12-14 18:51:44.338,2022-12-14 18:51:44.338,0
...,...,...,...,...,...,...,...,...
147,6387cfbe119842ac5d7dd34b,F4:12:FA:E2:24:04,05/02/10,"[{'macAddress': 'F6:12:FA:E2:06:18', 'strength...",NaN,2022-11-30 21:48:46.262,2022-11-30 21:48:46.262,0
148,6387cfc5119842ac5d7dd384,F4:12:FA:E2:24:04,05/02/10,"[{'macAddress': 'F6:12:FA:E2:06:18', 'strength...",NaN,2022-11-30 21:48:53.858,2022-11-30 21:48:53.858,0
149,6387cfcd119842ac5d7dd3bb,F4:12:FA:E2:24:04,05/02/10,"[{'macAddress': 'F6:12:FA:E2:06:18', 'strength...",NaN,2022-11-30 21:49:01.409,2022-11-30 21:49:01.409,0
150,6387cfd4119842ac5d7dd3f1,F4:12:FA:E2:24:04,05/02/10,"[{'macAddress': 'F6:12:FA:E2:06:18', 'strength...",NaN,2022-11-30 21:49:08.933,2022-11-30 21:49:08.933,0


## Transformando a coluna signals em colunas com o nome como o MacAddress e o valor como a potência para conseguir treinar a inteligência

In [8]:
formatedTotalSignalsDf = pd.DataFrame()
for index, row in df.iterrows():
  signalsDf = pd.DataFrame(row.signals)
  signalsDf = signalsDf.drop('_id', axis=1)
  macAddressArray = signalsDf['macAddress']
  strengthArray = signalsDf['strength']

  formatedDfData = {}
  for signalsIndex, signalsRow in signalsDf.iterrows():
    formatedDfData[signalsRow.macAddress] = signalsRow.strength 

  formatedSignalsDf = pd.DataFrame(formatedDfData, [index])
  
  formatedTotalSignalsDf = pd.concat([formatedTotalSignalsDf, formatedSignalsDf])
 
df = pd.concat([df, formatedTotalSignalsDf], axis=1)

,_id,deviceId,room,signals,battery,createdAt,updatedAt,__v,F6:12:FA:E2:55:04,FE:F6:3A:68:98:A3,...,FC:5C:45:C0:5F:68,FC:5C:45:C0:5F:69,FC:5C:45:00:5F:6A,F6:12:FA:E2:3F:24,FC:5C:45:C0:55:13,8A:29:A4:23:08:AC,F6:12:FA:E2:06:18,96:00:DA:76:2A:CD,EE:4A:23:4A:FB:C9,6E:C7:EC:79:19:D0
0,639a1a8b639dabb65f7a1fd9,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'F6:12:FA:E2:55:04', 'strength...",60.0,2022-12-14 18:48:43.522,2022-12-14 18:48:43.522,0,-46.0,-52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,639a1afe639dabb65f7a1ffd,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'FE:F6:3A:68:98:A3', 'strength...",60.0,2022-12-14 18:50:38.387,2022-12-14 18:50:38.387,0,NaN,-46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,639a1b13639dabb65f7a201a,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'FE:F6:3A:68:98:A3', 'strength...",60.0,2022-12-14 18:50:59.473,2022-12-14 18:50:59.473,0,NaN,-56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,639a1b24639dabb65f7a2045,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'FC:5C:45:00:57:48', 'strength...",60.0,2022-12-14 18:51:16.251,2022-12-14 18:51:16.251,0,NaN,-59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,639a1b40639dabb65f7a207a,F4:12:FA:E2:24:04,05/01/01,"[{'macAddress': 'FC:5C:45:00:57:48', 'strength...",60.0,2022-12-14 18:51:44.338,2022-12-14 18:51:44.338,0,NaN,-59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,6387cfbe119842ac5d7dd34b,F4:12:FA:E2:24:04,05/02/10,"[{'macAddress': 'F6:12:FA:E2:06:18', 'strength...",NaN,2022-11-30 21:48:46.262,2022-11-30 21:48:46.262,0,NaN,NaN,...,NaN,NaN,NaN,-86.0,-71.0,NaN,-45.0,-74.0,-87.0,NaN
148,6387cfc5119842ac5d7dd384,F4:12:FA:E2:24:04,05/02/10,"[{'macAddress': 'F6:12:FA:E2:06:18', 'strength...",NaN,2022-11-30 21:48:53.858,2022-11-30 21:48:53.858,0,NaN,NaN,...,NaN,NaN,NaN,-84.0,-73.0,NaN,-44.0,-73.0,-90.0,NaN
149,6387cfcd119842ac5d7dd3bb,F4:12:FA:E2:24:04,05/02/10,"[{'macAddress': 'F6:12:FA:E2:06:18', 'strength...",NaN,2022-11-30 21:49:01.409,2022-11-30 21:49:01.409,0,NaN,NaN,...,NaN,NaN,NaN,-76.0,-72.0,NaN,-44.0,-74.0,-93.0,-92.0
150,6387cfd4119842ac5d7dd3f1,F4:12:FA:E2:24:04,05/02/10,"[{'macAddress': 'F6:12:FA:E2:06:18', 'strength...",NaN,2022-11-30 21:49:08.933,2022-11-30 21:49:08.933,0,NaN,NaN,...,NaN,NaN,NaN,-78.0,-71.0,NaN,-47.0,-75.0,NaN,NaN


## Exclusão de colunas que não serão utilizadas para treinar o modelo

In [9]:
#Exclusão das colunas que não serão utilizadas
df = df.drop('signals', axis=1)
df = df.drop('createdAt', axis = 1)
id_column = df['_id']
df = df.drop('_id', axis=1)
df = df.drop('deviceId', axis=1)
df = df.drop('__v', axis = 1)
df = df.drop('updatedAt', axis = 1)
if 'battery' in df.columns:
  df = df.drop('battery', axis = 1)

In [10]:
#Preencher NaN com 0
df = df.fillna(0)

In [24]:
df.head(5)

,room,F6:12:FA:E2:55:04,FE:F6:3A:68:98:A3,FC:5C:45:C0:63:58,FC:5C:45:00:63:58,FC:5C:45:C0:63:59,FC:5C:45:00:57:48,FC:5C:45:C0:57:48,FC:5C:45:C0:57:49,FC:5C:45:00:57:4A,...,FC:5C:45:C0:5F:68,FC:5C:45:C0:5F:69,FC:5C:45:00:5F:6A,F6:12:FA:E2:3F:24,FC:5C:45:C0:55:13,8A:29:A4:23:08:AC,F6:12:FA:E2:06:18,96:00:DA:76:2A:CD,EE:4A:23:4A:FB:C9,6E:C7:EC:79:19:D0
0,05/01/01,-46.0,-52.0,-67.0,-68.0,-68.0,-69.0,-69.0,-70.0,-70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,05/01/01,0.0,-46.0,0.0,-79.0,-78.0,-65.0,-66.0,-66.0,-65.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,05/01/01,0.0,-56.0,-79.0,-78.0,-79.0,-61.0,-61.0,-61.0,-60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,05/01/01,0.0,-59.0,-76.0,-82.0,-76.0,-53.0,-53.0,-53.0,-53.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,05/01/01,0.0,-59.0,-80.0,-79.0,-80.0,-52.0,-52.0,-52.0,-52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Treinamento do modelo

Para fornecer a localização do dispositivo, foi escolhido o modelo de classificação RandomForest do scikit-learn.

In [12]:
#Aqui vamos dividir o treinamento e teste
from sklearn.model_selection import train_test_split

# Dividindo x e y
x = df.drop('room', axis=1)
y = df['room']

## Oversampling dos dados para balancear o dataset

In [13]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(random_state=42)
X_over, y_over = oversample.fit_resample(x, y)

## Dividindo a base de dados em set de treinamento e teste

In [14]:
# Dividindo dados para treino e dados para teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

## Treinando o modelo

In [16]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(criterion='entropy', random_state=42)
rfc.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=42)

## Predizendo a coluna room com o modelo já treinado

In [17]:
y_pred = rfc.predict(x_test)

array(['05/01/01', '05/02/10', '05/02/08', '05/01/02', '05/02/10',
       '05/02/10', '05/01/04', '05/02/06', '05/02/06', '05/02/06',
       '05/02/10', '05/02/08', '05/02/08', '05/02/06', '05/01/01',
       '05/01/01', '05/01/01', '05/02/09', '05/02/08', '05/02/08',
       '05/02/08', '05/02/09', '05/01/01', '05/01/01', '05/01/01',
       '05/02/10', '05/02/09', '05/01/02', '05/02/08', '05/03/01',
       '05/01/02'], dtype=object)

## Resultados do modelo

In [18]:
accuracy_score(y_test, y_pred)

1.0

In [19]:
precision_score(y_test, y_pred, average='micro')

1.0

In [20]:
recall_score(y_test, y_pred, average='micro')

1.0

In [21]:
!pip install -U -q PyDrive
!pip install shap

ERROR: unknown command "install shap" - maybe you meant "install"


#Exportação do modelo para a utilização em outro colab

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
from joblib import dump

dump(rfc, '/content/drive/MyDrive/Colab Notebooks/modelos/localizacao.model')

['/content/drive/MyDrive/Colab Notebooks/modelos/localizacao.model']